In [ ]:
using Compat
using QPControl
using RigidBodyDynamics
using AtlasRobot
using HumanoidLCMSim
using HumanoidLCMSim.AtlasSim
using JSExpr # FIXME: https://github.com/JunoLab/Blink.jl/issues/134

In [ ]:
# load URDF
const mechanism = AtlasSim.addflatground!(AtlasRobot.mechanism());

In [ ]:
# create optimizer
using MathOptInterface
using OSQP.MathOptInterfaceOSQP
const MOI = MathOptInterface
const optimizer = OSQPOptimizer()
MOI.set!(optimizer, OSQPSettings.Verbose(), false)
MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [ ]:
# create low level controller
const lowlevel = MomentumBasedController{4}(mechanism, optimizer);
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = RigidBodyDynamics.Contact.location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6 # TODO
        contact.weight[] = 1e-3
    end
end

In [ ]:
# create standing controller
info = AtlasSim.atlasrobotinfo(mechanism)
nominalstate = MechanismState(mechanism)
AtlasSim.initialize!(nominalstate, info)
pelvis = successor(info.floatingjoint, mechanism)
comref = center_of_mass(nominalstate) - FreeVector3D(root_frame(mechanism), 0., 0., 0.05)
const standingcontroller = StandingController(
    lowlevel, collect(values(info.feet)), pelvis, nominalstate, comref=comref);

In [ ]:
# create LCM control publisher
const publisher = LCMControlPublisher(info, standingcontroller;
    robot_state_channel="EST_ROBOT_STATE",
    robot_command_channel="ATLAS_COMMAND");

In [ ]:
# start listening for state messages and publishing command messages in response
handle(publisher, async=true);

In [ ]:
# run simulation
using HumanoidLCMSim
sol = AtlasSim.run(controlΔt = 1 / 500, final_time = 5.0);

In [ ]:
# checks
using Compat.Test
finalstate = MechanismState(mechanism)
copyto!(finalstate, sol.u[end])
@test sol.retcode == :Success
@test center_of_mass(finalstate) ≈ comref atol=2e-2
@test norm(velocity(finalstate)) ≈ 0 atol=5e-3